<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/logic/Fault_controller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Navigate to project folder and ensure we're on the 'develop' branch
import os, subprocess, sys

REPO_DIR = "/content/Cloud_Computing"
if not os.path.isdir(REPO_DIR):
    subprocess.run([
        "git", "clone", "-b", "develop",
        "https://github.com/Method-for-Software-System-Development/Cloud_Computing.git",
        REPO_DIR
    ], check=True)

# Update & checkout develop if repo already exists
subprocess.run(["git", "-C", REPO_DIR, "fetch", "origin"], check=True)
subprocess.run(["git", "-C", REPO_DIR, "checkout", "develop"], check=True)
subprocess.run(["git", "-C", REPO_DIR, "pull"], check=True)

# Install importnb (only needed once per session)
%pip install importnb --quiet
!pip install sseclient-py --quiet

# Add firebase folder to Python path and import its notebook as module
sys.path.append(os.path.join(REPO_DIR, "firebase"))  # for FireBase.ipynb
sys.path.append(REPO_DIR)                            # for FAULT_RULES.ipynb
from importnb import Notebook

with Notebook():
    import FireBase as fb                           # now fb.add_user, fb.get_user, etc. are available
    from FAULT_RULES import FAULT_RULES as fr       # now FAULT_RULES are aviable to use


firebase_url = fb.firebase_url

omer_cohen 200
shira_gold 200
daniel_bar 200
roni_benami 200
lior_levi 200
yael_elyashiv 200
noa_katz 200
matan_tal 200
or_peled 200
tom_segal 200
Indoor Sensor Data:
Status: 200 | Uploaded: {'Temperature': 26.4, 'Humidity': 41.0, 'Pressure': 974.1, 'Distance': 210.1}
Status: 200 | Uploaded: {'Temperature': 30.2, 'Humidity': 28.7, 'Pressure': 963.5, 'Distance': 8.9}
Status: 200 | Uploaded: {'Temperature': 36.5, 'Humidity': 91.3, 'Pressure': 919.8, 'Distance': 12.2}
Status: 200 | Uploaded: {'Temperature': 27.1, 'Humidity': 73.0, 'Pressure': 1050.4, 'Distance': 205.0}
Status: 200 | Uploaded: {'Temperature': 25.8, 'Humidity': 45.2, 'Pressure': 960.0, 'Distance': 210.0}
Status: 200 | Uploaded: {'Temperature': 32.7, 'Humidity': 29.5, 'Pressure': 940.2, 'Distance': 7.5}
Status: 200 | Uploaded: {'Temperature': 28.9, 'Humidity': 89.9, 'Pressure': 950.6, 'Distance': 250.0}
Status: 200 | Uploaded: {'Temperature': 30.0, 'Humidity': 70.1, 'Pressure': 960.1, 'Distance': 9.5}
Status: 200 | Uploaded:

In [4]:
import requests
import json
from datetime import datetime

# Create faults from all existing sensor data in Firebase (one pass, no threads)
def process_faults_from_database():
    active_faults_url = f"{firebase_url}faults/activeFaults.json"
    existing = requests.get(active_faults_url).json()
    existing_titles = set()
    for fault in (existing or {}).values():
        if fault and isinstance(fault, dict) and fault.get("status") == "Active":
            existing_titles.add(fault.get("title"))

    location_map = {
        "IndoorData": "Indoor",
        "OutdoorData": "Outdoor"
    }

    for folder, location in location_map.items():
        url = f"{firebase_url}SensorsData/{folder}.json"
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to load {folder}")
            continue

        data = response.json()
        if not isinstance(data, dict):
            print(f"No valid data in {folder}")
            continue

        for reading in data.values():
            if not isinstance(reading, dict):
                continue

            for sensor_type, value in reading.items():
                key = f"{sensor_type.lower()}_{location.lower()}"
                for rule in fr.get(key, []):
                    min_val = rule.get("min")
                    max_val = rule.get("max")

                    if (min_val is None or value >= min_val) and (max_val is None or value < max_val):
                        if rule["title"] in existing_titles:
                            print(f"Skipping duplicate active fault: {rule['title']}")
                            continue

                        title_key = f"{rule['title']} ({datetime.now().strftime('%d-%m-%Y %H:%M:%S')})"

                        fault = {
                            "sensor": rule["sensor"],
                            "severity": rule["severity"],
                            "title": rule["title"],
                            "value": value,
                            "status": "Active",
                            "repaired_by": "",
                            "timestamp": datetime.now().isoformat(),
                            "score": 50 if rule["severity"] == "Low" else 100 if rule["severity"] == "Medium" else 200,
                            "actions": rule["actions"]
                        }

                        requests.put(
                            f"{firebase_url}faults/activeFaults/{title_key}.json",
                            data=json.dumps(fault)
                        )

                        existing_titles.add(rule["title"])
                        print(f"[{location}] Fault Created: {rule['title']} (Severity: {rule['severity']})")

# Resolve a specific fault by moving it from active to resolved
def resolve_fault(fault_title: str, username: str):
    url = f"{firebase_url}faults/activeFaults/{fault_title}.json"
    r = requests.get(url)
    if r.status_code != 200 or r.json() is None:
        print("Fault not found or already resolved.")
        return False

    fault_data = r.json()
    fault_data["status"] = "Resolved"
    fault_data["repaired_by"] = username

    requests.put(
        f"{firebase_url}faults/resolvedFaults/{fault_title}.json",
        data=json.dumps(fault_data)
    )

    requests.delete(f"{firebase_url}faults/activeFaults/{fault_title}.json")

    print(f"Fault '{fault_title}' resolved by {username}")
    return True

process_faults_from_database()

[Indoor] Fault Created: Indoor Humidity below 30% (Severity: Medium)
[Indoor] Fault Created: Indoor Distance below 10 cm (Severity: High)
Skipping duplicate active fault: Indoor Humidity below 30%
[Indoor] Fault Created: Indoor Temperature between 30°C–35°C (Severity: Medium)
[Indoor] Fault Created: Indoor Humidity above 90% (Severity: High)
[Indoor] Fault Created: Indoor Pressure below 920 hPa (Severity: High)
[Indoor] Fault Created: Indoor Temperature above 35°C (Severity: High)
[Indoor] Fault Created: Indoor Humidity between 70%–90% (Severity: Medium)
[Indoor] Fault Created: Indoor Pressure between 1040–1060 hPa (Severity: Low)
Skipping duplicate active fault: Indoor Distance below 10 cm
Skipping duplicate active fault: Indoor Humidity below 30%
[Indoor] Fault Created: Indoor Pressure between 940–960 hPa (Severity: Low)
Skipping duplicate active fault: Indoor Temperature between 30°C–35°C
Skipping duplicate active fault: Indoor Distance below 10 cm
Skipping duplicate active fault: I